# Oject detection evaluation

Test a Detector on A Customized Dataset (here, I use the Faster RCNN)


In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode
import matplotlib.pyplot as plt
import cv2
import os
from detectron2.data.datasets import register_coco_instances

## Load the model

In [ ]:
# model path

# SwAV-FTAL-F4
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_SW_FTAL_F4/Train_2628/model_best_53.8845.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_SW_FTAL_F4/Train_2628/config.yaml"

# # SwAV-Scratch-F4
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_exp3/Freeze_4/SW_RN50_100e_scratch/SSL_train_2628/model_best_52.7624.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_exp3/Freeze_4/SW_RN50_100e_scratch/SSL_train_2628/config.yaml"

# # Baseline-F4
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_exp3/Freeze_4/SL_RN50_100e_FTAL/SL_train_2628/model_best_44.2123.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_exp3/Freeze_4/SL_RN50_100e_FTAL/SL_train_2628/config.yaml"




# SwAV-FTAL-F2
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models/SW_RN50_100e_FTAL/Self_train_224_tiles/SSL_train_2628/model_best_57.8298.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models/SW_RN50_100e_FTAL/Self_train_224_tiles/SSL_train_2628/config.yaml"

# # SwAV-Scratch-F2
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models/SW_RN50_100e_scratch/SSL_train_2628/model_best_58.5722.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models/SW_RN50_100e_scratch/SSL_train_2628/config.yaml"

# # Baseline-F2
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SL_models/FTAL_RN50_tiles_224/SL_train_2628/model_best_58.1664.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SL_models/FTAL_RN50_tiles_224/SL_train_2628/config.yaml"

# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SL_models/FTAL_RN50_tiles_224/SL_train_2628_1/model_best_58.8810.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SL_models/FTAL_RN50_tiles_224/SL_train_2628_1/config.yaml"

# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SL_models/FTAL_RN50_tiles_224/SL_train_2628_2/model_best_58.5027.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SL_models/FTAL_RN50_tiles_224/SL_train_2628_2/config.yaml"

model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/first_round_model_train/SW_FTAL_F4/SSL_train_2628/model_best_53.8845.pth"
model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/first_round_model_train/SW_FTAL_F4/SSL_train_2628/config.yaml"


cfg = get_cfg()
cfg.merge_from_file(model_config_path)
cfg.MODEL.WEIGHTS = model_checkpoint_path 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)


## Register my custom test dataset

dataset should be in COCO format

In [ ]:
# # Register GJO test dataset
register_coco_instances("GJO_test", {}, "/scratch/tjian/Data/GJO_SSL/tiles_224/SL_test/annotations/SL_test_2620.json", "/scratch/tjian/Data/GJO_SSL/tiles_224/SL_test/test/")
Test_Dataset_name="GJO_test"
Class_name=["litter"]


# Register Amsterdam test dataset
# register_coco_instances("AM_test", {}, "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Amsterdam/annotations/test.json", "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Amsterdam/images/")
# Test_Dataset_name="AM_test"
# Class_name=["litter"]

# Register Vietnam test dataset
# register_coco_instances("Vietnam_test", {}, "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Vietnam/annotations/SL_test_1091.json", "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Vietnam/test/")
# Test_Dataset_name="Vietnam_test"
# Class_name=["litter"]

# Register Groningen test dataset
# register_coco_instances("Groningen_test", {}, "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Groningen/annotations/SL_test_525.json", "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Groningen/test/")
# Test_Dataset_name="Groningen_test"
# Class_name=["litter"]

# # Register a test dataset
# register_coco_instances("test", {}, "/scratch/tjian/Data/GJO_SSL/test_1/test_PR/labels_coco/test.json", "/scratch/tjian/Data/GJO_SSL/test_1/test_PR/images/")
# Test_Dataset_name="test"
# Class_name=["litter"]

## Prediction

### (1) Predict one image

In [ ]:
im = cv2.imread("/scratch/tjian/Data/Jakarta_coco/val/Grogol_Location_1_2018-04-30 16-00-00-05_cam6.jpg")
outputs = predictor(im)
# print(outputs) # It will print num_instances, box location, scores, prediction labels

v = Visualizer(im,
               MetadataCatalog.get(Test_Dataset_name).set(thing_classes=Class_name),
               instance_mode=ColorMode.SEGMENTATION,
               scale=1.0
               )

out = v.draw_instance_predictions(outputs["instances"].to("cpu"))  # do not write "gpu"
img=out.get_image()
result=cv2.imwrite(r"/scratch/tjian/Data/Jakarta_coco/Predictions_SSL/predict.jpg", img)


### (2) Predict images in a folder

In [ ]:
import os

# define the folder path 

# (1) Amsterdam test folder
# folder_path = r"/scratch/tjian/Data/GJO_SSL/test/images/"

# (2) Vietnam test folder
# folder_path = r"/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Vietnam/test/"

# (3) Groningen test folder
# folder_path = r"/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Groningen/test/"

# (4) GJO test folder
# folder_path = r"/scratch/tjian/Data/GJO_SSL/tiles_224/images_only_litter_GJO/"

folder_path = r"/scratch/tjian/Data/Borgharen_test/BH-small-north/images/"


# define output folder of predicted images

# (1) Amsterdam test folder
# out_path = r"/scratch/tjian/Data/GJO_SSL/test/pred_images/"

# (2) Vietnam test folder
# out_path = r"/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SL_models/FTAL_RN50_tiles_224/SL_train_2628/Exp4/Vietnam_test_results/pred_images/"

# (3) Groningen test folder
# out_path = r"/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_exp3/Freeze_4/SL_RN50_100e_FTAL/SL_train_2628/Exp4/Groningen_test_results/pred_images/"

# (4) GJO test folder
# out_path = r"/scratch/tjian/Data/GJO_SSL/test 1/prediction_NMS_0.5/"
# out_path = r"/scratch/tjian/Data/GJO_SSL/tiles_224/GJO_pred_NMS_0.7/"

out_path = r"/scratch/tjian/Data/Borgharen_test/BH-small-north/predictions/"


file_names = os.listdir(folder_path)
for filename in file_names:
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            out_file_name= os.path.join(out_path, filename)
            im = cv2.imread(image_path)
            # start predicting
            outputs = predictor(im)
            print(outputs)
            print(len(outputs["instances"].get("pred_boxes")))
            v = Visualizer(im,
                           MetadataCatalog.get(Test_Dataset_name).set(thing_classes=Class_name),
                           instance_mode=ColorMode.SEGMENTATION,
                           scale=1.0
                           )
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))  # do not write "gpu"
            img=out.get_image()
            result=cv2.imwrite(out_file_name, img)

print("Done")


### (3) Test on a test dataset with labels

Run a model to predict images with ground-truth labels and output the accuracy, e.g., AP50

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

save_evaluate_file_dir = r"/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/first_round_model_train/SW_FTAL_F4/SSL_train_2628/test/"

# create the "save_evaluate_file_dir" if they do not exist
if not os.path.isdir(save_evaluate_file_dir):
    os.mkdir(save_evaluate_file_dir)

evaluator = COCOEvaluator(Test_Dataset_name, output_dir=save_evaluate_file_dir)
val_loader = build_detection_test_loader(cfg, Test_Dataset_name)
print(inference_on_dataset(predictor.model, val_loader, evaluator))

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# define the test dataset path

# save the results from GJO test dataset
# save_evaluate_file_dir = r"/scratch/tjian/Data/GJO_SSL/test_1/test_PR/SSL_pred_results_NMS_0.5/"

# save_evaluate_file_dir = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_F2/SW_RN50_100e_scratch/SSL_train_124/test_results/"

# save the results from Amsterdam test dataset
# save_evaluate_file_dir = r"/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SL_models/FTAL_RN50_tiles_224/SL_train_2628_5/Exp4/AM_test_results/"

# save the results from Vietnam test dataset
# save_evaluate_file_dir = r"/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SL_models/FTAL_RN50_tiles_224/SL_train_2628_5/Exp4/Vietnam_test_results/"

# save the results from Groningen test dataset
save_evaluate_file_dir = r"/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/first_round_model_train/SW_FTAL_F4/SSL_train_2628/test/"


# create the "save_evaluate_file_dir" if they do not exist
if not os.path.isdir(save_evaluate_file_dir):
    os.mkdir(save_evaluate_file_dir)


evaluator = COCOEvaluator(Test_Dataset_name, output_dir=save_evaluate_file_dir)
val_loader = build_detection_test_loader(cfg, Test_Dataset_name)
print(inference_on_dataset(predictor.model, val_loader, evaluator))

### (4) Output accuracy according to prediction and ground-truth files

Compare predictions file (json file) and ground-truth files (json file), and output the accuracy, e.g., AP50

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Load ground truth annotations
coco_gt = COCO('/scratch/tjian/Data/GJO_SSL/test_1/test_PR/labels_coco/test.json')

# Load detection results
# coco_dt = coco_gt.loadRes('/scratch/tjian/Data/GJO_SSL/test_1/test_PR/SSL_pred_results_NMS_0.5/coco_instances_results.json')

coco_dt = coco_gt.loadRes('/scratch/tjian/Data/GJO_SSL/test_1/test_PR/SSL_pred_results_NMS_0.1/coco_instances_results.json')

# coco_dt = coco_gt.loadRes('/scratch/tjian/Data/GJO_SSL/test_1/test_PR/BL_pred_results_NMS_0.5/coco_instances_results.json')


# Create COCOEval object
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')

# Run evaluation
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# Get precision and recall values at different IoU thresholds
precision = coco_eval.eval['precision']
recall = coco_eval.eval['recall']

### (5) Output confusion matrix on test dataset with annotations, i.e., TP, FP, FN

See the "main_Confusion_matrix_OD.ipynb"


### (6) Calculate the FP on a dataset without litter annotations

In [ ]:
import os

# define the folder path 


# (2) Vietnam test folder
# folder_path = r"/scratch/tjian/Data/GJO_SSL/tiles_224/images_no_litter_Vietnam/"

# (3) Groningen test folder
folder_path = r"/scratch/tjian/Data/GJO_SSL/tiles_224/images_no_litter_Groningen/"

# (4) GJO test folder
# folder_path = r"/scratch/tjian/Data/GJO_SSL/tiles_224/images_no_litter_GJO/"


file_names = os.listdir(folder_path)
FP = 0
for filename in file_names:
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            im = cv2.imread(image_path)
            # start predicting
            outputs = predictor(im)
            # print(outputs)
            # print(len(outputs["instances"].get("pred_boxes")))
            FP = FP + len(outputs["instances"].get("pred_boxes"))

print("FP: ", FP)


In [ ]:
### (6) Output confusion matrix on test dataset with annotations, i.e., TP, FP, FN

# This code only works for **"Single class"**;

# Labels: The COCO form of bbox annotations (in json file) is (Xmin, Ymax, width, height)

# Output: Detetcion bbox form: (Xmin, Ymax, width, height)

# Refer to: https://stackoverflow.com/questions/71475164/build-confusion-matrix-for-instance-segmantation-mask-r-cnn-from-detectron2

# ######################
# import tools.confusion_matrix as CM

# GJO test dataset
# test_annotations_path = "/scratch/tjian/Data/GJO_SSL/tiles_224/SL_test/annotations/SL_test_2620.json"
# test_images_path = "/scratch/tjian/Data/GJO_SSL/tiles_224/SL_test/test/"

# Amsterdam test dataset
# test_annotations_path = "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Amsterdam/annotations/test.json"
# test_images_path = "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Amsterdam/images/"

# Vietnam test dataset
# test_annotations_path = "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Vietnam/annotations/SL_test_1091.json"
# test_images_path = "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Vietnam/test/"

# Groningen test dataset
# test_annotations_path = "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Groningen/annotations/SL_test_525.json"
# test_images_path = "/scratch/tjian/Data/GJO_SSL/tiles_224/Exp4/Groningen/test/"

# load model

# # SwAV-FTAL-F4
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_exp3/Freeze_4/SW_RN50_100e_FTAL/SSL_train_2628/model_best_53.8845.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_exp3/Freeze_4/SW_RN50_100e_FTAL/SSL_train_2628/config.yaml"

# SwAV-scratch-F4
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_exp3/Freeze_4/SW_RN50_100e_scratch/SSL_train_2628/model_best_52.7624.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_exp3/Freeze_4/SW_RN50_100e_scratch/SSL_train_2628/config.yaml"

# # Baseline-F4
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_exp3/Freeze_4/SL_RN50_100e_FTAL/SL_train_2628/model_best_44.2123.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models_exp3/Freeze_4/SL_RN50_100e_FTAL/SL_train_2628/config.yaml"



# SwAV-FTAL-F2
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models/SW_RN50_100e_FTAL/Self_train_224_tiles/SSL_train_2628/model_best_57.8298.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models/SW_RN50_100e_FTAL/Self_train_224_tiles/SSL_train_2628/config.yaml"

# # SwAV-Scratch-F2
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models/SW_RN50_100e_scratch/SSL_train_2628/model_best_58.5722.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SSL_models/SW_RN50_100e_scratch/SSL_train_2628/config.yaml"

# # Baseline-F2
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SL_models/FTAL_RN50_tiles_224/SL_train_2628/model_best_58.1664.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_plastic_SSL/checkpoints/train_weights/SL_models/FTAL_RN50_tiles_224/SL_train_2628/config.yaml"



# TP, FP, FN = CM.confusion_matrix(model_config_path,
#                                 model_checkpoint_path,
#                                 test_annotations_path, 
#                                 test_images_path)
# print("TP:", TP)
# print("FP:", FP)
# print("FN:", FN)
